In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [3]:
from shapely.geometry import Point
def intpt_func(row):
    return Point(row['INTPTLON'], row['INTPTLAT'])

In [4]:
#loading LODES data

ham_lodes = pd.read_csv('./hamilton_lodes_2019.csv', dtype={"TRACTCE20_home":"string", "TRACTCE20_work":"string"})
ham_lodes.h_geocode = ham_lodes.h_geocode.apply(lambda x: int(x/1000))
ham_lodes.w_geocode = ham_lodes.w_geocode.apply(lambda x: int(x/1000))
ham_lodes.w_geocode = ham_lodes.w_geocode.astype(str)
ham_lodes.h_geocode = ham_lodes.h_geocode.astype(str)

#loading Hamilton county geodata
ham_cbg = pd.read_csv('./ham_cbg.csv')
ham_cbg['intpt'] = ham_cbg[['INTPTLAT', 'INTPTLON']].apply(lambda p: intpt_func(p), axis=1)
ham_cbg = gpd.GeoDataFrame(ham_cbg, geometry=gpd.GeoSeries.from_wkt(ham_cbg.geometry))
ham_cbg.GEOID = ham_cbg.GEOID.astype(str)

c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [5]:
#loading residential buildings
res_build = pd.read_csv('./ham_residential_buildings2.csv', index_col=0)
res_build = gpd.GeoDataFrame(res_build, geometry=gpd.GeoSeries.from_wkt(res_build.geometry))
res_build['location'] = res_build.geometry.apply(lambda p: [p.y, p.x])

#loading work buildings
com_build = pd.read_csv('./work_loc_poi_com_civ.csv', index_col=0)
com_build = gpd.GeoDataFrame(com_build, geometry=gpd.GeoSeries.from_wkt(com_build.geometry))
com_build['location'] = com_build.geometry.apply(lambda p: [p.y, p.x])
com_build = com_build.reset_index()
com_build.GEOID = com_build.GEOID.astype(str)

#loading all buildings (MS dataset)
ms_build = pd.read_csv('./ham_buildings_MS.csv')
ms_build = gpd.GeoDataFrame(ms_build, geometry=gpd.GeoSeries.from_wkt(ms_build.geo_centers))
ms_build.GEOID = ms_build.GEOID.astype(str)
ms_build['location'] = ms_build.geometry.apply(lambda p: [p.y, p.x])

In [6]:
ham_lodes = ham_lodes.groupby(['h_geocode', 'w_geocode']).agg(total_jobs=('total_jobs', sum)).reset_index().merge(ham_cbg[['GEOID', 'geometry']], left_on='h_geocode', right_on='GEOID').rename({'geometry':'home_geom'}, axis=1).drop('GEOID', axis=1).merge(ham_cbg[['GEOID', 'geometry']], left_on='w_geocode', right_on='GEOID').rename({'geometry':'work_geom'}, axis=1).drop('GEOID', axis=1).sort_values('total_jobs', ascending=False).reset_index(drop=True)
ham_lodes = gpd.GeoDataFrame(ham_lodes)

In [7]:
ham_lodes_homes = ham_lodes.drop('work_geom', axis=1).rename({'home_geom':'geometry'}, axis=1)
ham_lodes_work = ham_lodes.drop('home_geom', axis=1).rename({'work_geom':'geometry'}, axis=1)
ham_lodes_work.head()

,h_geocode,w_geocode,total_jobs,geometry
0,470650113214,470650004001,133,"POLYGON ((-85.27190 35.04104, -85.27182 35.041..."
1,470650004001,470650004001,100,"POLYGON ((-85.27190 35.04104, -85.27182 35.041..."
2,470650113231,470650004001,90,"POLYGON ((-85.27190 35.04104, -85.27182 35.041..."
3,470650113142,470650004001,84,"POLYGON ((-85.27190 35.04104, -85.27182 35.041..."
4,470650104353,470650004001,82,"POLYGON ((-85.27190 35.04104, -85.27182 35.041..."


In [8]:
res_build_cbg = res_build[['osmid', 'geometry']].sjoin(ham_lodes_homes)
com_build_cbg = com_build[['GEOID', 'geometry']].sjoin(ham_lodes_work)

In [9]:
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

In [98]:
#generating array of start and return times (in 15 min intervals)
from datetime  import datetime, timedelta
times_morning = [datetime.strptime(dt.strftime('%H:%M'), '%H:%M') for dt in 
       datetime_range(datetime(2016, 9, 1, 7), datetime(2016, 9, 1, 9, 10), 
       timedelta(minutes=15))]
times_evening = [datetime.strptime(dt.strftime('%H:%M'), '%H:%M') for dt in 
       datetime_range(datetime(2016, 9, 1, 16), datetime(2016, 9, 1, 18, 10), 
       timedelta(minutes=15))]

In [11]:
res_build.GEOID = res_build.GEOID.astype(str)
com_build.GEOID = com_build.GEOID.astype(str)

In [109]:
import random
import tqdm
from tqdm.notebook import tqdm_notebook

#setting the random seed
np.random.seed(42)
random.seed(42)

prob_matrix = gpd.GeoDataFrame()
for idx, movement in tqdm_notebook(ham_lodes.iterrows(), total=ham_lodes.shape[0]):

    res = res_build[res_build.GEOID == movement.h_geocode].reset_index(drop=True)
    if res.empty:
        res = ms_build[ms_build.GEOID == movement.h_geocode].sample(n=movement.total_jobs, random_state=42).reset_index(drop=True)

    com = com_build[com_build.GEOID == movement.w_geocode].reset_index(drop=True)
    if com.empty:
        com = ms_build[ms_build.GEOID == movement.w_geocode].sample(n=movement.total_jobs, random_state=42).reset_index(drop=True)
        
    r = res
    c = com
   
    for job in range(movement.total_jobs):
     
        if c.empty:
            c = com
        if r.empty:
            r = res

        rand_r = random.randrange(0, r.shape[0])
        rand_c = random.randrange(0, c.shape[0])
        r_df = r.iloc[rand_r]
        c_df = c.iloc[rand_c]
        r = r.drop([rand_r]).reset_index(drop=True)
        c = c.drop([rand_c]).reset_index(drop=True)
        
        time_slot1 = np.random.choice(times_morning, size=1, replace=True)
        time_slot2 = np.random.choice(times_evening, size=1, replace=True)

        temp = gpd.GeoDataFrame()

        temp.loc[job, 'h_geocode'] = movement.h_geocode
        temp.loc[job, 'w_geocode'] = movement.w_geocode
        temp.loc[job, 'total_jobs'] = movement.total_jobs
        temp.loc[job, 'home_loc_lat'] = r_df.location[0]
        temp.loc[job, 'home_loc_lon'] = r_df.location[1]
        temp.loc[job, 'work_loc_lat'] = c_df.location[0]
        temp.loc[job, 'work_loc_lon'] = c_df.location[1]
        temp.loc[job, 'go_time'] = time_slot1[0]
        temp.loc[job, 'go_time_str'] = time_slot1[0].strftime('%H:%M')
        temp.loc[job, 'return_time'] = time_slot2[0]
        temp.loc[job, 'return_time_str'] = time_slot2[0].strftime('%H:%M')

        prob_matrix = prob_matrix.append(temp, ignore_index=True)

  0%|          | 0/14282 [00:00<?, ?it/s]

In [110]:
def func_home_pt(row):
    return Point(row.home_loc_lon, row.home_loc_lat)
def func_work_pt(row):
    return Point(row.work_loc_lon, row.work_loc_lat)

In [111]:
prob_matrix['home_geom'] = prob_matrix[['home_loc_lat', 'home_loc_lon']].apply(lambda row: func_home_pt(row), axis=1)
prob_matrix['work_geom'] = prob_matrix[['work_loc_lat', 'work_loc_lon']].apply(lambda row: func_work_pt(row), axis=1)
prob_matrix.h_geocode = prob_matrix.h_geocode.astype(str)
prob_matrix.w_geocode = prob_matrix.w_geocode.astype(str)

c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [113]:
prob_matrix.head(20)

,h_geocode,w_geocode,total_jobs,home_loc_lat,home_loc_lon,work_loc_lat,work_loc_lon,go_time,go_time_str,return_time,return_time_str,home_geom,work_geom
0,470650113214,470650004001,133.0,35.028112,-85.143854,35.050563,-85.290914,1900-01-01 08:30:00,08:30,1900-01-01 16:45:00,16:45,POINT (-85.1438535 35.0281115),POINT (-85.290914 35.050563)
1,470650113214,470650004001,133.0,35.030310,-85.142181,35.048659,-85.289930,1900-01-01 08:45:00,08:45,1900-01-01 17:00:00,17:00,POINT (-85.1421810823235 35.03030972249326),POINT (-85.28993 35.048659)
2,470650113214,470650004001,133.0,35.025655,-85.139637,35.048659,-85.289930,1900-01-01 08:30:00,08:30,1900-01-01 16:30:00,16:30,POINT (-85.13963685165518 35.02565500814558),POINT (-85.28993 35.048659)
3,470650113214,470650004001,133.0,35.034019,-85.144271,35.045573,-85.278477,1900-01-01 08:30:00,08:30,1900-01-01 17:45:00,17:45,POINT (-85.14427076896568 35.03401899788406),POINT (-85.278477 35.045573)
4,470650113214,470650004001,133.0,35.035347,-85.144450,35.046745,-85.286959,1900-01-01 08:00:00,08:00,1900-01-01 16:45:00,16:45,POINT (-85.14445042705833 35.035346692943534),POINT (-85.286959 35.046745)
5,470650113214,470650004001,133.0,35.027357,-85.144265,35.042761,-85.276122,1900-01-01 08:45:00,08:45,1900-01-01 17:45:00,17:45,POINT (-85.14426478583007 35.02735708111118),POINT (-85.276122 35.042761)
6,470650113214,470650004001,133.0,35.023359,-85.141712,35.048659,-85.289930,1900-01-01 07:30:00,07:30,1900-01-01 17:15:00,17:15,POINT (-85.14171206028338 35.02335906613595),POINT (-85.28993 35.048659)
7,470650113214,470650004001,133.0,35.023553,-85.142581,35.047930,-85.291408,1900-01-01 08:00:00,08:00,1900-01-01 16:15:00,16:15,POINT (-85.14258140967239 35.023552820352606),POINT (-85.291408 35.04793)
8,470650113214,470650004001,133.0,35.035496,-85.143452,35.044040,-85.271541,1900-01-01 08:45:00,08:45,1900-01-01 17:15:00,17:15,POINT (-85.14345214500128 35.03549564456316),POINT (-85.271541 35.04404)
9,470650113214,470650004001,133.0,35.027471,-85.141958,35.049479,-85.275772,1900-01-01 07:15:00,07:15,1900-01-01 17:00:00,17:00,POINT (-85.14195833391271 35.027471396798965),POINT (-85.275772 35.049479)


In [115]:
prob_matrix.to_csv('lodes_combinations.csv', index=False)